$\Huge Code$ $\hspace{0.1cm}$ $\Huge to$ $\hspace{0.1cm}$ $\Huge draw$ $\hspace{0.1cm}$ $\Huge uniformly$ $\hspace{0.1cm}$ $\Huge sampled$ $\hspace{0.1cm}$ $\Huge tuples$ $\hspace{0.1cm}$ $\Huge of$ $\hspace{0.1cm}$ $\Huge coordinates$ $\hspace{0.1cm}$ $\Huge on$ $\hspace{0.1cm}$ $\Huge the$ $\hspace{0.1cm}$ $\Huge sphere$ $\Huge :$ 

# Modules : 

In [12]:
%matplotlib inline
import healpy as hp
import matplotlib.pyplot as plt 
from matplotlib import rc
rc('text', usetex=True)
from astropy.io import fits
import numpy as np
from astropy import constants as cst
from astropy.cosmology import FlatLambdaCDM
import pysm as pysm
import ccatp_sky_model as sky
import os.path
from astropy.coordinates import SkyCoord

In [13]:
cosmo = FlatLambdaCDM(H0=70, Om0=0.3, Tcmb0=2.7255)
T_CMB = cosmo.Tcmb0.si.value
k_B = cst.k_B.value
h = cst.h.value
c = cst.c.value

In [14]:
os_path = "/vol/arc3/data1/sz/CCATp_sky_model/templates/masks/"

# Function : 

In [15]:
def sample_sphere_uniform(n, mask = None, radec = True):

	'''Draws uniformly sampled tuples of coordinates on the sphere. 
	All-sky masks in the healpix format can be applied, in which case 
	masked areas will be excluded.
	
	Parameters
	----------
	n: int
		Number of data points to be drawn
	mask: float array, optional
		All-sky healpix mask. If a mask is used data points will 
		only be drawn in areas that are not masked. If mask is set
		to 'advACT', 'SPT', 'Dust', or 'NVSS', the respective 
		survey masks will be used. Default: None
	radec: bool, optional
		Determines the coordinate system of the output. If True, 
		equatorial coordinates will be returned, i.e. RA, DEC (fk5). 
		If False, galactic coordinates are returned. Default: True
		
	Returns
	-------
	phi, theta
		longitude and latitude of sampled points in equatorial or 
		galactic coordinate system
	'''
    
	if mask is None:
		phi = 360 * np.random.random(n)
		theta = np.arccos(2*np.random.random(n) - 1)*180/np.pi - 90
	else:
		if mask == 'advACT':
			mask = hp.read_map(os_path + 'Adv_ACT_survey_mask.fits', dtype = np.int16)  
		elif mask == 'SPT':
			mask = hp.read_map(os_path + 'SPT-SZ_survey_mask.fits', dtype = np.int16)
		elif mask == 'Dust':
			mask = hp.read_map(os_path + 'galactic_dust_mask.fits', dtype = np.int16)
		elif mask == 'NVSS':
			mask = hp.read_map(os_path + 'galactic_dust_nvss_mask.fits', dtype = np.int16)
	
		nside = hp.get_nside(mask)

		phi = np.zeros(n)
		theta = np.zeros(n)

		i = int(0)
		while i < n:
			phi_guess = 360 * np.random.random()
			theta_guess = np.arccos(2*np.random.random() - 1)*180/np.pi - 90

			index = hp.ang2pix(nside, phi_guess, theta_guess, lonlat = True)
			
			if mask[index] != 0:
				phi[i] = phi_guess
				theta[i] = theta_guess
				i += 1

	if radec is True:
		print('output will be fk5 coordinates (RA, DEC) for the equinox J2000')
		c = SkyCoord(phi, theta, frame='galactic', unit='deg')
		c_fk5 = c.transform_to('fk5')
		phi = c_fk5.ra.degree
		theta = c_fk5.dec.degree
	else:
		print('output will be galactic coordinates (glon, glat)')

	return(phi, theta)

# Launch : 

In [21]:
A = sample_sphere_uniform(n=1, mask = 'Dust', radec = True)

NSIDE = 256
ORDERING = RING in fits file
INDXSCHM = IMPLICIT
output will be fk5 coordinates (RA, DEC) for the equinox J2000


In [22]:
print(A)

(array([30.23595674]), array([-65.91914277]))


In [23]:
B = sample_sphere_uniform(n=10, mask = 'Dust', radec = True)

NSIDE = 256
ORDERING = RING in fits file
INDXSCHM = IMPLICIT
output will be fk5 coordinates (RA, DEC) for the equinox J2000


In [24]:
print(B)

(array([ 24.27843545, 143.49017644,  15.32096072, 218.83830286,
       253.00300038, 147.05960007, 219.7140473 , 326.20804537,
       110.35721382, 178.3705408 ]), array([ 42.38941674,  61.58932736,  -9.24416142,  53.03156   ,
        13.99566365, -13.46815802,  -8.05135892,  20.28448773,
        64.94176461,   6.08697253]))


In [39]:
C = [i[0] for i in B]

In [40]:
print(C)

[24.278435446684032, 42.389416738079255]
